In [2]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=f96e242e499c6046f389b3120a8527149993c9547478b7ce4c9000c33e72e1c6
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=99a4e2bbb4418f5cc377a3d5e140dd0e573a96015035c8494e8c20cb243222c2
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [27]:
import pandas as pd
from sklearn.manifold import TSNE
from umap import UMAP
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import plotly.offline as pyo

In [4]:
from google.colab import drive
drive.mount('/content/drive')
vertices_path = '/content/drive/MyDrive/map_generalization/Vertices_Labels'

Mounted at /content/drive


In [5]:
california_df = pd.read_parquet(f'{vertices_path}/California.parq')
florida_df = pd.read_parquet(f'{vertices_path}/Florida.parq')
idaho_df = pd.read_parquet(f'{vertices_path}/idaho.parq')
louisiana_df = pd.read_parquet(f'{vertices_path}/Louisiana.parq')
maine_df = pd.read_parquet(f'{vertices_path}/Maine.parq')
northcarolina_df = pd.read_parquet(f'{vertices_path}/NorthCarolina.parq')
texas_df = pd.read_parquet(f'{vertices_path}/Texas.parq')

In [6]:
def add_state_name_to_col(df, state_name):
    df['state'] = state_name
    return df

In [7]:
california_df = add_state_name_to_col(california_df, 'California')
florida_df = add_state_name_to_col(florida_df, 'Florida')
idaho_df = add_state_name_to_col(idaho_df, 'Idaho')
louisiana_df = add_state_name_to_col(louisiana_df, 'Louisiana')
maine_df = add_state_name_to_col(maine_df, 'Maine')
northcarolina_df = add_state_name_to_col(northcarolina_df, 'North Carolina')
texas_df = add_state_name_to_col(texas_df, 'Texas')

In [8]:
print(f"No of points in California is: {california_df.shape}")
print(f"No of points in Florida is: {florida_df.shape}")
print(f"No of points in Idaho is: {idaho_df.shape}")
print(f"No of points in Louisiana is: {louisiana_df.shape}")
print(f"No of points in Maine is: {maine_df.shape}")
print(f"No of points in North Carolina is: {northcarolina_df.shape}")
print(f"No of points in Texas is: {texas_df.shape}")

No of points in California is: (186383, 12)
No of points in Florida is: (1087787, 12)
No of points in Idaho is: (35244, 13)
No of points in Louisiana is: (494846, 12)
No of points in Maine is: (294372, 12)
No of points in North Carolina is: (518948, 12)
No of points in Texas is: (336584, 12)


In [48]:
concatenated_df = pd.concat([california_df, idaho_df, louisiana_df])
features = concatenated_df[['Longitude', 'Latitude', 'len_forward', 'len_backward', 'angle', 'offset',
                           'ratio1', 'ratio2']]
features

,Longitude,Latitude,len_forward,len_backward,angle,offset,ratio1,ratio2
0,-117.212186,32.775095,78.952213,12.728309,0.000000,0.000000e+00,0.109734,-0.000000e+00
1,-117.211686,32.773995,57.527132,78.952213,20.923366,2.183531e+08,1.008215,1.613041e+06
2,-117.212157,32.774460,52.356821,57.527132,158.746826,2.898872e+08,0.962930,2.540326e+06
3,-117.212486,32.775195,32.526329,52.356821,164.381949,2.845860e+08,0.654295,2.193641e+06
4,-117.212742,32.775503,20.037115,32.526329,155.695381,2.833410e+08,0.498574,2.687541e+06
...,...,...,...,...,...,...,...,...
494841,-89.673971,29.867069,0.672046,1.893691,156.789540,6.331520e+01,0.025924,6.397206e-01
494842,-89.673977,29.867107,2.339647,0.672046,166.190181,3.450687e+01,0.026703,3.059580e-01
494843,-89.673998,29.867157,2.002652,2.339647,174.173455,1.439302e+01,0.036610,1.213479e-01
494844,-89.674016,29.867190,1.112434,2.002652,135.000104,8.904259e+01,0.042319,1.209659e+00


In [49]:
features.isna().sum()

Longitude       0
Latitude        0
len_forward     2
len_backward    0
angle           4
offset          0
ratio1          0
ratio2          0
dtype: int64

In [50]:
features_na_drop = features.dropna()
features_na_drop

,Longitude,Latitude,len_forward,len_backward,angle,offset,ratio1,ratio2
0,-117.212186,32.775095,78.952213,12.728309,0.000000,0.000000e+00,0.109734,-0.000000e+00
1,-117.211686,32.773995,57.527132,78.952213,20.923366,2.183531e+08,1.008215,1.613041e+06
2,-117.212157,32.774460,52.356821,57.527132,158.746826,2.898872e+08,0.962930,2.540326e+06
3,-117.212486,32.775195,32.526329,52.356821,164.381949,2.845860e+08,0.654295,2.193641e+06
4,-117.212742,32.775503,20.037115,32.526329,155.695381,2.833410e+08,0.498574,2.687541e+06
...,...,...,...,...,...,...,...,...
494840,-89.673988,29.867002,1.893691,0.232804,164.605402,6.953051e+01,0.028068,9.177401e-01
494841,-89.673971,29.867069,0.672046,1.893691,156.789540,6.331520e+01,0.025924,6.397206e-01
494842,-89.673977,29.867107,2.339647,0.672046,166.190181,3.450687e+01,0.026703,3.059580e-01
494843,-89.673998,29.867157,2.002652,2.339647,174.173455,1.439302e+01,0.036610,1.213479e-01


In [32]:
tsne = TSNE(n_components=2, random_state=0)

In [ ]:
projections = tsne.fit_transform(features_na_drop)

In [51]:
umap_2d = UMAP(n_components=2, init='random', random_state=0)

In [52]:
umap_projections = umap_2d.fit_transform(features_na_drop)

In [60]:
umap_projections

array([[ 8.615752 ,  7.338771 ],
       [19.526777 ,  5.3838315],
       [-4.545338 ,  4.4737167],
       ...,
       [ 1.8497392, 12.689225 ],
       [ 8.744985 ,  7.7228   ],
       [ 4.519261 ,  8.915727 ]], dtype=float32)

In [54]:
np.save('california_idaho_louisiana_umap.npy', umap_projections)

In [55]:
color_mapping = {'California': 0, 'Idaho': 1, 'Louisiana': 2}
concatenated_df['color'] = concatenated_df['state'].map(color_mapping)

In [56]:
fig_2d = go.Figure()

fig_2d.add_trace(go.Scatter(
    x=umap_projections[:, 0],
    y=umap_projections[:, 1],
    mode='markers',
    name='Embeddings',
    marker=dict(
        color=concatenated_df['color'],
        colorscale='Viridis',
    )
))

Output hidden; open in https://colab.research.google.com to view.

In [46]:
fig_2d = px.scatter(
    umap_projections, x=0, y=1,
    color=concatenated_df.color[:-1], labels={'color': 'state'}
)

In [57]:
pyo.plot(fig_2d, filename='california_idaho__louisiana_umap.html')

'california_idaho__louisiana_umap.html'